In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Instructions
rubric={points}

You will earn points for following these instructions and successfully submitting your work on Gradescope.  

### Group wotk instructions

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2.
  
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).
- If you would like to use late tokens for the homework, all group members must have the necessary late tokens available. Please note that the late tokens will be counted for all members of the group.   


### General submission instructions

- Please **read carefully
[Use of Generative AI policy](https://ubc-cs.github.io/cpsc330-2025W1/syllabus.html#use-of-generative-ai-in-the-course)** before starting the homework assignment. 
- **Run all cells before submitting:** Go to `Kernel -> Restart Kernel and Clear All Outputs`, then select `Run -> Run All Cells`. This ensures your notebook runs cleanly from start to finish without errors.
  
- **Submit your files on Gradescope.**  
   - Upload only your `.ipynb` file **with outputs displayed** and any required output files.
     
   - Do **not** submit other files from your repository.  
   - If you need help, see the [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/).  
- **Check that outputs render properly.**  
   - Make sure all plots and outputs appear in your submission.
     
   - If your `.ipynb` file is too large and doesn't render on Gradescope, also upload a PDF or HTML version so the TAs can view your work.  
- **Keep execution order clean.**  
   - Execution numbers must start at "1" and increase in order.
     
   - Notebooks without visible outputs may not be graded.  
   - Out-of-order or missing execution numbers may result in mark deductions.  
- **Follow course submission guidelines:** Review the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html) for detailed guidance on completing and submitting assignments. 
   
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
words=["wise","good","cry","cat"]

In [7]:
glove_wiki_vectors.most_similar("wise")

[('good', 0.6905286908149719),
 ('honest', 0.6366483569145203),
 ('foolish', 0.6309534311294556),
 ('smart', 0.6156166195869446),
 ("'m", 0.6144938468933105),
 ('happy', 0.6134263873100281),
 ('very', 0.6130837798118591),
 ('intelligent', 0.6081452369689941),
 ('really', 0.6079776883125305),
 ('thing', 0.6052514910697937)]

In [8]:
glove_wiki_vectors.most_similar("good")

[('better', 0.893191397190094),
 ('sure', 0.8314563035964966),
 ('really', 0.8297762274742126),
 ('kind', 0.8288268446922302),
 ('very', 0.8260800838470459),
 ('we', 0.8234356045722961),
 ('way', 0.8215398192405701),
 ('think', 0.8205099105834961),
 ('thing', 0.8171301484107971),
 ("'re", 0.8141680955886841)]

In [9]:
glove_wiki_vectors.most_similar("cry")

[('shout', 0.7581712603569031),
 ('scream', 0.7551398873329163),
 ('laugh', 0.750495970249176),
 ('cries', 0.729505181312561),
 ('crying', 0.693906843662262),
 ('hear', 0.6756482124328613),
 ('!', 0.6745465993881226),
 ('remember', 0.6657224893569946),
 ('cheer', 0.6600813865661621),
 ('wonder', 0.659145176410675)]

In [10]:
glove_wiki_vectors.most_similar("cat")

[('dog', 0.8798074722290039),
 ('rabbit', 0.7424426674842834),
 ('cats', 0.732300341129303),
 ('monkey', 0.7288709878921509),
 ('pet', 0.7190139889717102),
 ('dogs', 0.7163873314857483),
 ('mouse', 0.6915251016616821),
 ('puppy', 0.6800068616867065),
 ('rat', 0.6641026735305786),
 ('spider', 0.6501135230064392)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
for pair in word_pairs:
    print("The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], glove_wiki_vectors.similarity(pair[0], pair[1])))

#got format help from chap17 code!!

The similarity between coast and shore is 0.700
The similarity between clothes and closet is 0.546
The similarity between old and new is 0.643
The similarity between smart and intelligent is 0.755
The similarity between dog and cat is 0.880
The similarity between tree and lawyer is 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
for word in test_words:
    if word in glove_wiki_vectors:
        print(word, True)
    else:
        print(word, False)

covididiot False
fomo False
frenemies True
anthropause False
photobomb False
selfie False
pxg False
pacg False
cct True
escc True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

np.float32(0.14283238)

In [18]:
glove_wiki_vectors.similarity("white", "success")

np.float32(0.351824)

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
analogy("man", "violence", "woman")

man : violence :: woman : ?


,Analogy word,Score
0,conflict,0.658255
1,bloodshed,0.656648
2,clashes,0.654472
3,unrest,0.644717
4,violent,0.641396
5,harassment,0.621601
6,escalating,0.609073
7,tensions,0.601112
8,incidents,0.600471
9,sex,0.594911


In [20]:
analogy("woman", "violence", "man")

woman : violence :: man : ?


,Analogy word,Score
0,violent,0.752113
1,bloodshed,0.720317
2,fighting,0.692572
3,sectarian,0.690636
4,attacks,0.682866
5,clashes,0.678012
6,unrest,0.668297
7,killings,0.653631
8,terrorism,0.652779
9,conflict,0.650526


In [21]:
analogy("man", "ceo", "woman")

man : ceo :: woman : ?


,Analogy word,Score
0,executive,0.734881
1,cfo,0.635496
2,chairwoman,0.616618
3,chairman,0.602085
4,vice,0.589005
5,chief,0.570874
6,co-founder,0.557430
7,directors,0.543935
8,parent,0.539751
9,founder,0.539111


In [22]:
analogy("women", "ceo", "men")

women : ceo :: men : ?


,Analogy word,Score
0,executive,0.770698
1,chief,0.726417
2,chairman,0.678495
3,manager,0.643115
4,boss,0.636348
5,officer,0.636127
6,cfo,0.620644
7,executives,0.611107
8,founder,0.604095
9,managing,0.594499


In [23]:
glove_wiki_vectors.similarity("computer", "women")

np.float32(0.22772118)

In [24]:
glove_wiki_vectors.similarity("computer", "men")

np.float32(0.2443928)

In [25]:
glove_wiki_vectors.similarity("strong", "women")

np.float32(0.44901067)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

_There are worrisome biases in these embeddings trained on Wikipedia. For example, when I test man : ceo :: woman : ?, the model returns "parent." But when I try women : ceo :: men : ? No such word appeared, and instead the words are all good authority words. This showcases these embeddings still associating women with family, which is a strong stereotype. Moreover, when I try to test man : violence :: woman : ?, it gives words like "sex" and "harassment," which showcases that these embeddings also try to victimize or sexualize women while men are just associated with simple violent words like "terrorism" and "attacks."_

_An example of how using embeddings with biases could cause harm in the real world would be resume screening, and when the model is trying to screen the women's resumes, it will consider many qualified women candidates' good traits as "parent," which is a strong bias._

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [26]:
from sklearn.datasets import fetch_20newsgroups

In [27]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [28]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [29]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 61.6 MB/s  0:00:00 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [30]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [31]:
def clean_text(text):

    doc=nlp(text)

    tokens=[]

    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"]

    for token in doc:

        if token.like_url:
            tokens.append("URL")
            continue
        if token.like_email:
            tokens.append("EMAIL")
            continue
        if token.like_num:
            tokens.append("NUM")
            continue
        if token.is_stop or token.is_punct or token.is_space:
            continue
        if not token.is_alpha:
            continue
        if token.pos_ in irrelevant_pos:
            continue
            
        lemma = token.lemma_.lower()

        if len(lemma)<=3:
            continue
            
        tokens.append(lemma)

    return " ".join(tokens)

In [32]:
df["text_clean"] = df["text"].apply(clean_text)

In [33]:
df.iloc[2:6]

,text,target,target_name,text_clean
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have NUM newsgroup...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem appreciate help create NUM wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late foreign ministry spokesman ferhat ataman ...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder NUM ...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 4

_The most reasonable n_components should be 10. This is because when we set n_components=10, it provides the clearest and organized themes for each topic, unlike setting it equal to 3/6/15. The topic themes are politics, sports, Israel-Palestine, computers etc._

In [34]:
from sklearn.decomposition import LatentDirichletAllocation

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(df["text_clean"])

In [36]:
### best model set n_components=10
lda_ten=LatentDirichletAllocation(n_components=10,learning_method="batch", random_state=42)
lda_ten.fit(X)

import mglearn
mglearn.tools.print_topics(
    topics=range(10),
    feature_names=np.array(vec.get_feature_names_out()),
    sorting= np.argsort(lda_ten.components_, axis=1)[:, ::-1],
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
num           num           team          suck          num           
jesus         url           player        stay          firearm       
people        email         think         bronx         people        
church        file          militia       anas          crime         
know          image         right         blow          weapon        
believe       available     time          manhattan     control       
think         program       henrik        sink          fire          
bible         version       want          beauchaine    question      
come          system        hockey        queens        year          
christ        server        good          think         kill          


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      --------      --------      
num 

In [37]:
lda_six=LatentDirichletAllocation(n_components=6,learning_method="batch", random_state=42)
lda_six.fit(X)

import mglearn
mglearn.tools.print_topics(
    topics=range(6),
    feature_names=np.array(vec.get_feature_names_out()),
    sorting= np.argsort(lda_six.components_, axis=1)[:, ::-1],
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
num           num           num           atheist       num           
know          game          people        believe       people        
people        team          right         think         armenian      
jesus         play          israel        exist         know          
think         year          state         argument      armenians     
believe       player        think         belief        turkish       
church        season        time          religion      come          
come          good          weapon        people        tell          
time          hockey        israeli       evidence      time          
good          league        know          thing         year          


topic 5       
--------      
num           
url           
file          
email         
program       
image         
window        
available  

In [38]:
lda_fifteen=LatentDirichletAllocation(n_components=15,learning_method="batch", random_state=42)
lda_fifteen.fit(X)

import mglearn
mglearn.tools.print_topics(
    topics=range(15),
    feature_names=np.array(vec.get_feature_names_out()),
    sorting= np.argsort(lda_fifteen.components_, axis=1)[:, ::-1],
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
num           num           henrik        suck          num           
jesus         url           team          belief        rate          
church        email         plane         blow          number        
bible         file          armenia       bronx         homicide      
believe       image         love          sink          crime         
know          available     azeris        stay          greek         
people        version       armenians     queens        year          
christ        program       want          beauchaine    handgun       
faith         server        conflict      manhattan     study         
christians    system        think         work          kill          


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      --------      --------      
num 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 5

_Topic 0 : "Christian Religion"._

_Topic 1 : "Computer"._

_Topic 2 : "Hockey"._

_Topic 3 : "New York Slang"._

_Topic 4 : "Firearms and Crime"._

_Topic 5 : "Technical Programming"._

_Topic 6 : "Sports Discussions"._

_Topic 7 : "Philosophy Ideas and Words"._

_Topic 8 : "Armenian and Turkish"._

_Topic 9 : "Israel"._


In [39]:
import mglearn
mglearn.tools.print_topics(
    topics=range(10),
    feature_names=np.array(vec.get_feature_names_out()),
    sorting= np.argsort(lda_ten.components_, axis=1)[:, ::-1],
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
num           num           team          suck          num           
jesus         url           player        stay          firearm       
people        email         think         bronx         people        
church        file          militia       anas          crime         
know          image         right         blow          weapon        
believe       available     time          manhattan     control       
think         program       henrik        sink          fire          
bible         version       want          beauchaine    question      
come          system        hockey        queens        year          
christ        server        good          think         kill          


topic 5       topic 6       topic 7       topic 8       topic 9       
--------      --------      --------      --------      --------      
num 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [40]:
topics = lda_ten.transform(X)
pd.DataFrame(topics[:5])

,0,1,2,3,4,5,6,7,8,9
0,0.001887,0.001887,0.001887,0.001887,0.626614,0.001887,0.001887,0.001887,0.358287,0.001888
1,0.001250,0.001250,0.001250,0.001250,0.001250,0.717618,0.133681,0.139949,0.001250,0.001250
2,0.096621,0.299156,0.001695,0.001695,0.271510,0.001695,0.001695,0.322542,0.001695,0.001695
3,0.105389,0.004765,0.004762,0.004762,0.004763,0.544099,0.188242,0.004763,0.133693,0.004762
4,0.002565,0.002565,0.002564,0.002564,0.002564,0.002564,0.002564,0.002564,0.470884,0.508601


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

_1. Content-based filtering will predict ratings for new items and new users based on the content on items they liked before, and we will create a user profile for each user._

_2. The first negative consequence would be it could keep recommending users the items they already bought or items they are not interested in anymore, which causes users to get bored of such old and repeated recommendations. Thus, diversity and freshness of the recommender system are very important. The second negative consequence is the polarization of people’s thoughts. For example, if users like posts or videos that match their current views, the recommender system will keep showing them similar content, preventing exposure to new perspectives and ultimately increasing polarization._

_3. The transfer learning in NLP is to download a pre-trained model and fine-tune it for my specific task, instead of training a model from scratch._

<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top. 

Here is a quick checklist before submitting: 

- [ ] Restart kernel, clear outputs, and run all cells from top to bottom.  
- [ ] `.ipynb` file runs without errors and contains all outputs.  
- [ ] Only `.ipynb` and required output files are uploaded (no extra files).  
- [ ] Execution numbers start at **1** and are in order.  
- [ ] If `.ipynb` is too large and doesn't render on Gradescope, also upload a PDF/HTML version.  
- [ ] Reviewed the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html).  

![](img/eva-well-done.png)